# Import Statements

In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional
import torch.optim as optim
from torch.utils.data import TensorDataset, random_split, DataLoader
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

 # Read & Process Data

In [ ]:
data = pd.read_csv ('C:/Users/user/Desktop/Car_Price_Data.csv') # Read Data From CSV Using Pandas

data = data.drop(columns=['Car Model']) # Drop Car Model Since It Is Not Important To Prediction
data = data.drop(columns=['Car Make']) # Drop Car Make Since It Is Not Important To Prediction

if 'Engine Size (L)' in data.columns:
    data = data[pd.to_numeric(data['Engine Size (L)'], errors='coerce').notnull()] # Drop Any Engine Sizes That Are Electric Or Hybrid

data.dropna(inplace=True)

# Extract Features And Output And Scale Them And Turn Them To Tensor Objects

In [ ]:
X = data.drop(['Price (in USD)'], axis = 1) # These Are The Features
for col in X.columns:
    if X[col].dtype == 'object':
        X[col] = X[col].str.replace(',', '').astype(float) # Turn Everything To Type Float
y = data['Price (in USD)'].str.replace(',', '').astype(float) # This Is The Output And We Turned Everything To Type Float

scaler = StandardScaler() 
X = scaler.fit_transform(X) # Scale Features

scaler_y = StandardScaler()
y = scaler_y.fit_transform(y.values.reshape(-1, 1)).flatten()  # Scale Output

X_tensor = torch.FloatTensor(X) # Turn Features To Tensor
y_tensor = torch.FloatTensor(y) # Turn Output To Tensor

dataset = TensorDataset(X_tensor, y_tensor) # Create Tensor Dataset That Contains Both The Features And The Output

# Split The Dataset Into Training, Test, And Validation

In [ ]:
train_size = int(0.8 * len(data)) # Training Set Size
dev_size = int(0.1 * len(data)) # Validation Set Size
test_size = len(data) - train_size - dev_size # Test Set Size

train_data, dev_data, test_data = random_split(dataset, [train_size, dev_size, test_size]) # Split And Shuffle Data

batch_size = 64 # Batch Size

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True) # Load Training Data
dev_loader = DataLoader(dev_data, batch_size=batch_size, shuffle=False) # Load Validation Data
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False) # Load Testing Data

# Our Neural Network Class

In [ ]:
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(5, 512) # Input Layer
        self.bn1 = nn.BatchNorm1d(512) # Normalize Input Layer
        self.fc2 = nn.Linear(512, 256) # Hidden Layer 1
        self.bn2 = nn.BatchNorm1d(256) # Normalize Hidden Layer 1
        self.fc3 = nn.Linear(256, 128) # Hidden Layer 2
        self.bn3 = nn.BatchNorm1d(128) # Normalize Hidden Layer 2
        self.fc4 = nn.Linear(128, 64) # Hidden Layer 3
        self.bn4 = nn.BatchNorm1d(64) # Normalize Hidden Layer 3
        self.fc5 = nn.Linear(64, 32) # Hidden Layer 4
        self.bn5 = nn.BatchNorm1d(32) # Normalize Hidden Layer 4
        self.fc6 = nn.Linear(32, 1) # Output Layer

    def forward(self, x):
        x = torch.nn.functional.leaky_relu(self.bn1(self.fc1(x))) # Forward Propogation On Input Layer
        x = torch.nn.functional.leaky_relu(self.bn2(self.fc2(x))) # Forward Propogation On Hidden Layer 1
        x = torch.nn.functional.leaky_relu(self.bn3(self.fc3(x))) # Forward Propogation On Hidden Layer 2
        x = torch.nn.functional.leaky_relu(self.bn4(self.fc4(x))) # Forward Propogation On Hidden Layer 3
        x = torch.nn.functional.leaky_relu(self.bn5(self.fc5(x))) # Forward Propogation On Hidden Layer 4
        x = self.fc6(x) # Output
        return x

# Run The Model

In [ ]:
model = SimpleNN() # Initialize Model
criterion = nn.MSELoss() # Initialize Criterion
optimizer = optim.SGD(model.parameters(), lr=0.0001) # Initialize Gradient Descent

epochs = 150 # Number Of Epochs
for epoch in range(epochs):
    model.train() # Set Model To Train Mode
    running_loss = 0.0 # Initialize Loss

    for features, labels in train_loader:
        features, labels = features.float(), labels.float().unsqueeze(1) # Convert Features And Labels To Float
        optimizer.zero_grad() # Clear Gradients
        outputs = model(features) # Forward Pass
        loss = criterion(outputs, labels) # Compute Loss
        loss.backward() # Backward Pass
        optimizer.step() # Update Parameters
        running_loss += loss.item() # Add Loss

    print(f'Epoch [{epoch+1}/{epochs}], Training Loss: {running_loss/len(train_loader):.4f}') # Print Training Loss Per Epoch

# Evaluate Model

In [ ]:
def evaluate_model(loader):
    model.eval() # Set Model To Evaluate Mode
    mse_loss = 0.0 # Initialize Loss
    with torch.no_grad():
        for features, labels in loader:
            features, labels = features.float(), labels.float().unsqueeze(1) # Convert Features And Labels To Float
            outputs = model(features) # Forward Pass
            loss = criterion(outputs, labels) # Compute Loss
            mse_loss += loss.item() # Add Loss
    return mse_loss / len(loader) # Return Average MSE Over All Batch

# Calculate & Print Validation & Test MSE

In [ ]:
validation_mse = evaluate_model(dev_loader)
print(f'Validation MSE: {validation_mse:.4f}')

test_mse = evaluate_model(test_loader)
print(f'Test MSE: {test_mse:.4f}')